In [50]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from density import mach_parker, parker_critical
from EUV_flux import euv_flux_photons
from ifront_guess import surface_density

In [51]:
from astropy.constants import M_sun, R as R_gas, G, m_p, h
k = 1e-2 * u.cm**2/u.g # opacity
alpha_B = 2.6e-13 * (u.cm**3)/u.s # cm3s-1 at (T = 10^4 K) From Owen and Alvarez 2016

## Theoretical prediction

See Owen and Alvarez 2016

In [52]:
# Parameter list
T_H = 3500 * u.K # temperature in hydrostatic region
T_P= 1e4 * u.K # temperature in parker wind region

mu_H = 1 * u.g / u.mol # mean molecular weight in units of hydrogen mass
mu_P = mu_H/2

m_dot = 0.1
M_s = 2.107142857142857*M_sun
R_s = 0.4359452463077387 * u.astronomical_unit

M_BH = 4e6 * M_sun
d = 0.05 * u.parsec
a_H = np.sqrt(R_gas*T_H/mu_H)
a_P = np.sqrt(R_gas*T_P/mu_P)

rho_s = surface_density(a_H, M_s, R_s) # density at photosphere

In [53]:
#Find total photon flux
J_0 = euv_flux_photons(m_dot, M_BH)/(4*np.pi*d**2)
J_0.decompose()

/Users/jacopouggeri/opt/anaconda3/lib/python3.9/site-packages/astropy/units/quantity.py:611: RuntimeWarning: overflow encountered in exp
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/jacopouggeri/opt/anaconda3/lib/python3.9/site-packages/astropy/units/quantity.py:611: RuntimeWarning: overflow encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


<Quantity 1.26306732e+22 1 / (m2 s)>

In [54]:
H = min(R_s/3, a_H**2*R_s**2/(2*G*M_s))
H.to(u.astronomical_unit)

<Quantity 0.0014793 AU>

In [55]:
n = np.sqrt(J_0/(alpha_B*H))
n.to(1/u.cm**3)

<Quantity 1.48161685e+10 1 / cm3>

In [56]:
print((n*m_p).to(u.g/u.cm**3))

2.478184822220809e-14 g / cm3


In [57]:
R_c = parker_critical(a_P, M_s).to(u.astronomical_unit)

In [58]:
r = (R_s/R_c).value
print("Radius of star", R_s)
print("Radius of star squared", R_s**2)
print("Mach number", mach_parker(r))
M_dot =  2*np.pi*R_s**2*m_p*n*a_P*mach_parker(r)
print("Density:", (n*m_p/2).to("g/cm3"))
print("MLR:", M_dot.to(u.M_sun/u.a))

Radius of star 0.4359452463077387 AU
Radius of star squared 0.19004825777831494 AU2
Mach number 4.714437399512633e-09
Density: 1.2390924111104045e-14 g / cm3
MLR: 6.389815203583648e-16 solMass / a


In [59]:
from density import density_profile, parker_critical, mach_parker
from ifront_guess import ionisation_front, ionisation_fraction_error, ionisation_fraction_plot

In [60]:
def mass_loss_rate(r_I, R_s, M_s, a_H, a_P):
    R_I = r_I*parker_critical(a_P, M_s)
    rho_s = surface_density(a_H, M_s, R_s)
    rho = density_profile(r_I, r_I, R_s, M_s, rho_s, a_H, a_P)
    M_dot = 4*np.pi*(R_I**2)*a_P*mach_parker(r_I)*rho
    return M_dot

In [61]:
# Find mass loss rate for single case
r_I = ionisation_front(m_dot, M_s, M_BH, R_s, a_H, a_P, d)
print("Ionisation front squared:", (r_I*parker_critical(a_P, M_s).to(u.astronomical_unit))**2 )
print("Mach number:", mach_parker(r_I))
print("Density:", density_profile(r_I, r_I, R_s, M_s, rho_s, a_H, a_P).to("g/cm3"))
print("MLR:", mass_loss_rate(r_I, R_s, M_s, a_H, a_P).to(u.solMass/u.a))

Ionisation front squared: 0.21569452230362762 AU2
Mach number: 2.0196934310310468e-08
Density: 4.7019636905801047e-14 g / cm3
MLR: 1.1789480686300408e-14 solMass / a
